In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import activations
from tensorflow.keras.layers import Layer, Input, Embedding, LSTM, Dense, Attention,Flatten
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, f1_score

In [3]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_units):
        super(Encoder, self).__init__()
        # Embedding Layer
        self.embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)
        # Encode LSTM Layer
        self.encoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True, name="encode_lstm")
        
    def call(self, inputs):
        encoder_embed = self.embedding(inputs)
        encoder_outputs, state_h, state_c = self.encoder_lstm(encoder_embed)
        return encoder_outputs, state_h, state_c
        #return encoder_embed

In [4]:
class Decoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_units):
        super(Decoder, self).__init__()
        # Embedding Layer
        self.embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)
        # Decode LSTM Layer
        self.decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True, name="decode_lstm")
        # Attention Layer
        self.attention = Attention()
    
    def call(self, enc_outputs, dec_inputs, states_inputs):
    #def call(self, dec_inputs):
        decoder_embed = self.embedding(dec_inputs)
        dec_outputs, dec_state_h, dec_state_c = self.decoder_lstm(decoder_embed, initial_state=states_inputs)
        #attention_output = self.attention([dec_outputs, enc_outputs])
        return dec_outputs, dec_state_h, dec_state_c
        #return attention_output, dec_state_h, dec_state_c

In [5]:
def Seq2Seq(maxlen, embedding_dim, hidden_units, vocab_size):
    """
    seq2seq model
    """
    # Input Layer
    encoder_inputs = Input(shape=(maxlen,), name="encode_input")
    decoder_inputs = Input(shape=(maxlen,), name="decode_input")
    # Encoder Layer
    encoder = Encoder(vocab_size, embedding_dim, hidden_units)
    enc_outputs, enc_state_h, enc_state_c = encoder(encoder_inputs)
    dec_states_inputs = [enc_state_h, enc_state_c]
    # Decoder Layer
    decoder = Decoder(vocab_size, embedding_dim, hidden_units)
    attention_output, dec_state_h, dec_state_c = decoder(enc_outputs, decoder_inputs, dec_states_inputs)
    # Dense Layer
    flatten_outputs = Flatten()(attention_output)
    dense_outputs = Dense(vocab_size, activation='softmax', name="dense")(flatten_outputs)
    # seq2seq model
    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=dense_outputs)
    return model

In [6]:
maxlen = 82
embedding_dim = 50
hidden_units = 128
vocab_size = 3

model = Seq2Seq(maxlen, embedding_dim, hidden_units, vocab_size)
#model.build(input_shape=(10000))
#model.summary()

In [7]:
data = pd.read_csv('train/is_train_20190730.txt')

In [8]:
#use_cols = [i for i in train.columns if i not in ['link', 'label', 'current_slice_id', 'future_slice_id', 'label_pred']]
use_cols = [i for i in data.columns if i not in ['link', 'label']]

In [9]:
X_train = data[use_cols]
y_train = data['label']

In [10]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [11]:
X_train = (X_train-X_train.min(axis=0))/(X_train.max(axis=0)-X_train.min(axis=0))

In [12]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
model.compile(loss=loss_fn, optimizer='adam',metrics=['accuracy'])

In [13]:
class_weight={0:0.2,1:0.2,2:0.6}

In [18]:
model.fit([X_train, X_train], y_train, class_weight = class_weight,validation_data=([X_ver, X_ver], y_ver),
          batch_size=2048, epochs=3)

Epoch 1/3
250/250 [==============================] - 949s 4s/step - loss: 0.1164 - accuracy: 0.8345 - val_loss: 0.3831 - val_accuracy: 0.8627
Epoch 2/3
250/250 [==============================] - 943s 4s/step - loss: 0.0943 - accuracy: 0.8611 - val_loss: 0.3969 - val_accuracy: 0.8558
Epoch 3/3
250/250 [==============================] - 943s 4s/step - loss: 0.0927 - accuracy: 0.8615 - val_loss: 0.3597 - val_accuracy: 0.8661


In [19]:
import joblib
joblib.dump(model, 'tmd_.txt')

TypeError: cannot pickle '_thread.RLock' object

In [14]:
asd = pd.read_csv('train/is_train_20190701.txt')

In [15]:
X_ver = asd[use_cols]
y_ver = asd['label']

In [16]:
X_ver = np.array(X_ver)
y_ver = np.array(y_ver)

In [17]:
X_ver = (X_ver-X_ver.min(axis=0))/(X_ver.max(axis=0)-X_ver.min(axis=0))

In [65]:
res = model.predict([X_ver[0].reshape(1,-1),X_ver[0].reshape(1,-1)])
res

array([[0.8932275 , 0.09736255, 0.00940992]], dtype=float32)

In [20]:
res = model.predict([X_ver[:10000],X_ver[:10000]])

In [ ]:
for i in range(len(X_ver)):
    res = model.predict([X_ver[i:i+1],X_ver[i:i+1]])

In [22]:
temp = np.argmax(res,axis=1)

In [23]:
print(classification_report(y_ver[:10000], temp, digits=4))

              precision    recall  f1-score   support

           0     0.9032    0.9673    0.9342      8266
           1     0.6606    0.3757    0.4790      1352
           2     0.4921    0.4869    0.4895       382

    accuracy                         0.8690     10000
   macro avg     0.6853    0.6100    0.6342     10000
weighted avg     0.8547    0.8690    0.8556     10000



In [24]:
(0.4790+0.9342)*0.2+0.4895*0.6

0.57634

In [11]:
import gc
gc.collect()

1095